In [65]:
import pandas as pd
import numpy as np
import os


## Carga de datos
Se realiza carga de datos de los ficheros correspondientes, en formatos xls,csv, geojson y/o shp.


In [66]:
cwd = os.getcwd()
raw_dir = cwd + "/../Datos/Raw/"
processed_dir = cwd + "/../Datos/processed/"

In [67]:
# Cargar los datos de los restaurantes desde el archivo CSV
restaurantes = pd.read_csv(raw_dir + "RESTAURANTES KFC CALI.csv", delimiter=';')

In [68]:
# Cargar los datos de las transacciones desde el archivo CSV
transacciones = pd.read_csv( raw_dir + "pedidos.csv", delimiter=';', low_memory=False)


Se crea funcion para consultar cantidad de columnas y datos de cada dataset en el que se va a trabajar.

In [69]:
#conjunto de data para consultar
dataframes = [restaurantes, transacciones] 
#Se crea funcion npara imprimir tamaños de dataset se usara en varias ocasiones 
def imprimir_nombres_dataframes(dataframes):
    shapes = [df.shape for df in dataframes]  # Obtiene la forma de cada DataFrame
# Mostrar los nombres y formas de los DataFrames
    for i, df in enumerate(dataframes):
        shape = shapes[i]
        print(f"DataFrame {i + 1}:- Forma: {shape}")

In [70]:
#cantidad de datos 
#1. dataset: Restaurante
#2. Dataset: Transaccion
#3. Dataset: Mapas.
imprimir_nombres_dataframes(dataframes)

DataFrame 1:- Forma: (13, 4)
DataFrame 2:- Forma: (315924, 9)


## Visualización de datos.
Se visualiza en tablas casa dataseSet para idetificar como se compone la informacion.

In [71]:
#visualizacion de los datos de transacciones 
transacciones.head()

,10.125.38.1\MAXPOINT,CO-Rappi-274567881,45,2022-09-10 00:00:00.000,59300,NULL,NULL.1,Rappi,Rappi.1
0,10.125.38.1\MAXPOINT,CO-Rappi-274572835,45,2022-09-10 00:00:00.000,23800,NaN,NaN,Rappi,Rappi
1,10.125.38.1\MAXPOINT,CO-DiDi-5764618254974781692-5,45,2022-09-10 00:00:00.000,22900,3.0,-77.0,didi,didi
2,10.125.38.1\MAXPOINT,CO-Rappi-274584964,45,2022-09-10 00:00:00.000,53900,NaN,NaN,Rappi,Rappi
3,10.125.38.1\MAXPOINT,CO-Rappi-274589935,45,2022-09-10 00:00:00.000,15900,NaN,NaN,Rappi,Rappi
4,10.125.38.1\MAXPOINT,CO-Rappi-274610931,45,2022-09-10 00:00:00.000,63900,NaN,NaN,Rappi,Rappi


In [72]:
#visualizacion de los datos de transacciones 
restaurantes.head()

,nombre,rest,latitud,longitud
0,CIUDAD CORDOBA,187,3.396714,-76.509745
1,PALMETTO PLAZA,45,3.419646,-76.539767
2,AV SEXTA,61,3.465173,-76.530913
3,SAN FERNANDO,158,3.434722,-76.541968
4,PASO ANCHO,163,3.412630,-76.528697


Asignación de nombres a columnas del Dataset de Transacciones dado que se observa que no se cuenta con encabezados.
En el Dataset de restaurantes se renombra para ser mas sencillo el uso y ocupe menos espacio en pantalla

In [73]:
#encabezado de las columnas de los datos de transacciones.
encabezado_trans=['fuente', 'cod_trans', 'cod_rest','fecha_trans', 'valor_trans', 'latitud','longitud','canal','medio']  # Reemplaza 'data' con tus datos
transacciones.columns = encabezado_trans
transacciones.head(1)

,fuente,cod_trans,cod_rest,fecha_trans,valor_trans,latitud,longitud,canal,medio
0,10.125.38.1\MAXPOINT,CO-Rappi-274572835,45,2022-09-10 00:00:00.000,23800,NaN,NaN,Rappi,Rappi


In [74]:
#renombrar columnas de restaurante segun los datos.
restaurantes=restaurantes.rename(columns={
    'nombre': 'nom_rest',
    'rest':'cod_rest',
    'latitud':'latitud_rest',
    'longitud':'longitud_rest'
})
restaurantes.head(1)

,nom_rest,cod_rest,latitud_rest,longitud_rest
0,CIUDAD CORDOBA,187,3.396714,-76.509745


Descripción de datos, analsis de tipo de datos que se cuenta en los dataSet de transacciones y restaurantes, evaluación preliminar que datos se tiene.

In [75]:
# creamos funcion de analisis de completud y nulos en datos de restaurnates 
#y transacciones se usara en varias ocasiones 
def informacionDataFrame(c):
    cantidad_nulos_rest = c.isnull().sum()
    completud_rest = (c.shape[0] - cantidad_nulos_rest) / c.shape[0]
    resumen = pd.DataFrame({
        'Tipo de dato':c.dtypes,
        'Cantidad Nulos Rest': cantidad_nulos_rest,
        'Completud rest': completud_rest
    })
    print(resumen)

In [76]:
#mostrar tipos de datos y Evaluación de la calidad de los datos.
informacionDataFrame(restaurantes)
print('-'* 63)
informacionDataFrame(transacciones)

              Tipo de dato  Cantidad Nulos Rest  Completud rest
nom_rest            object                    0             1.0
cod_rest             int64                    0             1.0
latitud_rest       float64                    0             1.0
longitud_rest      float64                    0             1.0
---------------------------------------------------------------
            Tipo de dato  Cantidad Nulos Rest  Completud rest
fuente            object                    0        1.000000
cod_trans         object                    0        1.000000
cod_rest           int64                    0        1.000000
fecha_trans       object                    0        1.000000
valor_trans       object                    0        1.000000
latitud          float64               137738        0.564015
longitud         float64               137738        0.564015
canal             object                    0        1.000000
medio             object                    0        1.000

## Limpieza de datos
Se inicia con la eliminación de la columna ***fuente*** dado que esta no aporta al analisis.

In [77]:
#eliminacion de columna que no se usaran
transacciones = transacciones.drop('fuente', axis=1)
transacciones.head(1)

,cod_trans,cod_rest,fecha_trans,valor_trans,latitud,longitud,canal,medio
0,CO-Rappi-274572835,45,2022-09-10 00:00:00.000,23800,NaN,NaN,Rappi,Rappi


Vemos que hay columnas con tipo *"object"* por lo cual se asignar el tipo de dato correcto a cada columna.
se cambia puntuación en la columna *valor_trans*, asigna formato a *fecha_trans*.

In [78]:
#dar formato a la columna valor_trans.
transacciones['valor_trans'] = transacciones['valor_trans'].fillna(0)
transacciones['valor_trans'] = transacciones['valor_trans'].str.replace(',', '.')
transacciones['valor_trans'] = transacciones['valor_trans'].astype('float64')
transacciones['valor_trans'] = transacciones['valor_trans'].apply(lambda x: round(x) if pd.notnull(x) else x)

In [80]:
#dar formato a todo el dataset transacciones incluyento la fecha.
transacciones=transacciones.astype({
    'cod_trans':'string',
    'valor_trans':'int64',
    'canal':'string',
    'medio':'string'
})
#formato a la fechaS
transacciones["fecha_trans"] = pd.to_datetime(transacciones["fecha_trans"], format='%Y-%m-%d %H:%M:%S.%f')
transacciones["fecha_trans"] = pd.to_datetime(transacciones["fecha_trans"], format='%Y-%m-%d')

#dar formato a todo el dataset restaurantes.
restaurantes=restaurantes.astype({
    'nom_rest':'string',
    'cod_rest':'int64'
})
#mostrar tipos de datos y Evaluación de la calidad de los datos.
informacionDataFrame(transacciones)
transacciones.head(1)

               Tipo de dato  Cantidad Nulos Rest  Completud rest
cod_trans    string[python]                    0        1.000000
cod_rest              int64                    0        1.000000
fecha_trans  datetime64[ns]                    0        1.000000
valor_trans           int64                    0        1.000000
latitud             float64               137738        0.564015
longitud            float64               137738        0.564015
canal        string[python]                    0        1.000000
medio        string[python]                    0        1.000000


,cod_trans,cod_rest,fecha_trans,valor_trans,latitud,longitud,canal,medio
0,CO-Rappi-274572835,45,2022-09-10,23800,NaN,NaN,Rappi,Rappi


# _--------------------------------------------------------------------------------------

In [81]:
cod_rest_transacciones = transacciones['cod_rest'].unique()
cod_restaurantes = restaurantes['cod_rest'].unique()

# Compara los códigos de restaurante únicos en ambos dataframes
print(set(cod_rest_transacciones) - set(cod_restaurantes))  # Códigos en transacciones que no están en restaurantes
print(set(cod_restaurantes) - set(cod_rest_transacciones))  # Códigos en restaurantes que no están en transacciones



{24, 140}
set()


In [82]:
#unificar dataframe entre restaurantes y transacciones. 
#al unir se eliminan las transacciones asociadas a un cod_rest que no esten en Restaurantes.
df_unificado = pd.merge(transacciones, restaurantes, on='cod_rest')
print(df_unificado.shape)
informacionDataFrame(df_unificado)
df_unificado.head(5)

(274240, 11)
                 Tipo de dato  Cantidad Nulos Rest  Completud rest
cod_trans      string[python]                    0        1.000000
cod_rest                int64                    0        1.000000
fecha_trans    datetime64[ns]                    0        1.000000
valor_trans             int64                    0        1.000000
latitud               float64               120384        0.561027
longitud              float64               120384        0.561027
canal          string[python]                    0        1.000000
medio          string[python]                    0        1.000000
nom_rest       string[python]                    0        1.000000
latitud_rest          float64                    0        1.000000
longitud_rest         float64                    0        1.000000


,cod_trans,cod_rest,fecha_trans,valor_trans,latitud,longitud,canal,medio,nom_rest,latitud_rest,longitud_rest
0,CO-Rappi-274572835,45,2022-09-10,23800,NaN,NaN,Rappi,Rappi,PALMETTO PLAZA,3.419646,-76.539767
1,CO-DiDi-5764618254974781692-5,45,2022-09-10,22900,3.0,-77.0,didi,didi,PALMETTO PLAZA,3.419646,-76.539767
2,CO-Rappi-274584964,45,2022-09-10,53900,NaN,NaN,Rappi,Rappi,PALMETTO PLAZA,3.419646,-76.539767
3,CO-Rappi-274589935,45,2022-09-10,15900,NaN,NaN,Rappi,Rappi,PALMETTO PLAZA,3.419646,-76.539767
4,CO-Rappi-274610931,45,2022-09-10,63900,NaN,NaN,Rappi,Rappi,PALMETTO PLAZA,3.419646,-76.539767


In [83]:
# Asignar la latitud y longitud del restaurante cuando canal sea nulos
df_unificado['latitud'] = df_unificado['latitud'].fillna(df_unificado['latitud_rest'])
df_unificado['longitud'] = df_unificado['longitud'].fillna(df_unificado['longitud_rest'])

# Actualizar latitud y longitud cuando el medio sea "Didi"
df_unificado.loc[df_unificado['medio'] == 'didi', 'latitud'] = df_unificado.loc[df_unificado['medio'] == 'didi', 'latitud_rest'].values
df_unificado.loc[df_unificado['medio'] == 'didi', 'longitud'] = df_unificado.loc[df_unificado['medio'] == 'didi', 'longitud_rest'].values
df_unificado.head(2)

,cod_trans,cod_rest,fecha_trans,valor_trans,latitud,longitud,canal,medio,nom_rest,latitud_rest,longitud_rest
0,CO-Rappi-274572835,45,2022-09-10,23800,3.419646,-76.539767,Rappi,Rappi,PALMETTO PLAZA,3.419646,-76.539767
1,CO-DiDi-5764618254974781692-5,45,2022-09-10,22900,3.419646,-76.539767,didi,didi,PALMETTO PLAZA,3.419646,-76.539767


In [84]:
df_unificado['canal'] = df_unificado['canal'].str.lower()
porcentaje_comision = {'didi': 0.14, 'rappi': 0.145, 'web': 0.10, 'app': 0.11,'callcenter': 0.10,'tictuk': 0.12, 'ifood': 0.13}

df_unificado['comision'] = df_unificado.apply(lambda row: row['valor_trans'] * porcentaje_comision[row['canal']], axis=1)
df_unificado.head(10)

,cod_trans,cod_rest,fecha_trans,valor_trans,latitud,longitud,canal,medio,nom_rest,latitud_rest,longitud_rest,comision
0,CO-Rappi-274572835,45,2022-09-10,23800,3.419646,-76.539767,rappi,Rappi,PALMETTO PLAZA,3.419646,-76.539767,3451.0
1,CO-DiDi-5764618254974781692-5,45,2022-09-10,22900,3.419646,-76.539767,didi,didi,PALMETTO PLAZA,3.419646,-76.539767,3206.0
2,CO-Rappi-274584964,45,2022-09-10,53900,3.419646,-76.539767,rappi,Rappi,PALMETTO PLAZA,3.419646,-76.539767,7815.5
3,CO-Rappi-274589935,45,2022-09-10,15900,3.419646,-76.539767,rappi,Rappi,PALMETTO PLAZA,3.419646,-76.539767,2305.5
4,CO-Rappi-274610931,45,2022-09-10,63900,3.419646,-76.539767,rappi,Rappi,PALMETTO PLAZA,3.419646,-76.539767,9265.5
5,CO-DiDi-5764618393483282709-8,45,2022-09-10,53900,3.419646,-76.539767,didi,didi,PALMETTO PLAZA,3.419646,-76.539767,7546.0
6,CO-Rappi-274619730,45,2022-09-10,56400,3.419646,-76.539767,rappi,Rappi,PALMETTO PLAZA,3.419646,-76.539767,8178.0
7,CO-Rappi-274654880,45,2022-09-10,95700,3.419646,-76.539767,rappi,Rappi,PALMETTO PLAZA,3.419646,-76.539767,13876.5
8,CO-ifood-09101553034631481,45,2022-09-10,29900,3.419646,-76.539767,ifood,LOGISTIC,PALMETTO PLAZA,3.419646,-76.539767,3887.0
9,CO-ifood-09101628234632812,45,2022-09-10,31900,3.419646,-76.539767,ifood,LOGISTIC,PALMETTO PLAZA,3.419646,-76.539767,4147.0


Verificación de la informacion limpia y correcta sin nulos y completa. lista para usar.

In [85]:
#analisis de completud y nulos en datos de restaurnates y transacciones
informacionDataFrame(df_unificado)
#cantidad de datos 
#1. Dataset: Transaccion 
print(f"Dataset de transacciones:",df_unificado.shape)

                 Tipo de dato  Cantidad Nulos Rest  Completud rest
cod_trans      string[python]                    0             1.0
cod_rest                int64                    0             1.0
fecha_trans    datetime64[ns]                    0             1.0
valor_trans             int64                    0             1.0
latitud               float64                    0             1.0
longitud              float64                    0             1.0
canal          string[python]                    0             1.0
medio          string[python]                    0             1.0
nom_rest       string[python]                    0             1.0
latitud_rest          float64                    0             1.0
longitud_rest         float64                    0             1.0
comision              float64                    0             1.0
Dataset de transacciones: (274240, 12)


In [86]:
#exportar dataset listo para usar de transacciones
df_unificado.to_csv(processed_dir + 'datos_limpios_unidos.csv', index=False)